IMPORTACAO DA LIB

In [48]:
import duckdb

CONEXAO COM O BANCO

In [49]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

VENDO O QUE TEM NA TABLE bronze_z0019

In [50]:
df = con.execute("SELECT * FROM bronze_z0019").fetchdf()
df.head(10)


,NATBR,MAKTX,WEKS,MAINS,LABST,nome_arquivo,data_ingestao
0,1001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-09-10 10:38:07.280318
1,1002,MARTELO,BT50,100,1500,z0019_1.csv,2025-09-10 10:38:07.280318
2,1003,PREGO,BT10,100,50,z0019_1.csv,2025-09-10 10:38:07.280318
3,1004,SERRA,BT10,100,200,z0019_2.csv,2025-09-10 10:38:57.033303
4,1005,MACHADO,BT10,100,100,z0019_2.csv,2025-09-10 10:38:57.033303
5,1003,PREGO,BT10,100,60,z0019_2.csv,2025-09-10 10:38:57.033303


In [51]:
df = con.execute("""
               SELECT * 
                 FROM (
                        SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                        FROM bronze_z0019 
                        WHERE data_ingestao >= '2025-08-10'
                      ) WHERE row = 1
                """).fetchdf()
df.head(20)

,NATBR,MAKTX,WEKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,1002,MARTELO,BT50,100,1500,z0019_1.csv,2025-09-10 10:38:07.280318,1
1,1001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-09-10 10:38:07.280318,1
2,1003,PREGO,BT10,100,60,z0019_2.csv,2025-09-10 10:38:57.033303,1
3,1004,SERRA,BT10,100,200,z0019_2.csv,2025-09-10 10:38:57.033303,1
4,1005,MACHADO,BT10,100,100,z0019_2.csv,2025-09-10 10:38:57.033303,1


In [52]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])

df_final = df_final.rename(columns={"NATBR": "id"})
df_final = df_final.rename(columns={"MAKTX": "nm_produto"})
df_final = df_final.rename(columns={"WEKS": "id_categoria"})
df_final = df_final.rename(columns={"MAINS": "id_fornecedor"})
df_final = df_final.rename(columns={"LABST": "vl_preco"})


df_final.head(20)


,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,1002,MARTELO,BT50,100,1500
1,1001,PARAFUSO,BT10,100,100
2,1003,PREGO,BT10,100,60
3,1004,SERRA,BT10,100,200
4,1005,MACHADO,BT10,100,100


In [53]:
df2 = df_final


df2 = df2.astype(
                    {
                        'id': 'int', 
                        'nm_produto': 'str', 
                        'id_categoria': 'str', 
                        'id_fornecedor': 'int', 
                        'vl_preco': 'float'
                    }
                ) 

df2.dtypes

id                 int64
nm_produto        object
id_categoria      object
id_fornecedor      int64
vl_preco         float64
dtype: object

In [59]:
con.execute(""" 
                CREATE TABLE IF NOT EXISTS produtos (
                   id BIGINT,
                   nm_produto TEXT,
                   id_categoria TEXT,
                   id_fornecedor BIGINT,
                   vl_preco FLOAT
               )
            """)

In [60]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

# con.execute("DROP TABLE produtos")

In [61]:
df_result = con.execute("SELECT * FROM produtos").fetchdf()
df_result.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,1002,MARTELO,BT50,100,1500.0
1,1001,PARAFUSO,BT10,100,100.0
2,1003,PREGO,BT10,100,60.0
3,1004,SERRA,BT10,100,200.0
4,1005,MACHADO,BT10,100,100.0


In [62]:
con.close()